In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
rutor_url = 'http://5.45.84.133/new/'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

502

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [ ]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [ ]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)]
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
2,Зарубежные фильмы,16 Май 23,"Подземелья и драконы: Честь среди воров / Dungeons & Dragons: Honor Among Thieves (2023) WEB-DL 720p от селезень | P, A",5.54 GB,40,30
3,Зарубежные фильмы,16 Май 23,Смертельный выстрел / Dead Shot (2023) WEB-DLRip-AVC | Pazl Voice,1.59 GB,52,23
10,Зарубежные фильмы,16 Май 23,Смертельный выстрел / Dead Shot (2023) WEB-DLRip от New-Team | Pazl Voice,1.46 GB,65,31
11,Зарубежные фильмы,16 Май 23,Смертельный выстрел / Dead Shot (2023) WEB-DL 1080p от New-Team | Pazl Voice,5.14 GB,81,31
12,Зарубежные фильмы,16 Май 23,Air: Большой прыжок / Air (2023) WEB-DLRip-AVC от DoMiNo & селезень | P | HDRezka Studio,1.50 GB,67,42
13,Зарубежные фильмы,16 Май 23,"Air: Большой прыжок / Air (2023) UHD WEB-DL-HEVC 2160p | 4K | HDR | Dolby Vision Profile 8 | HDRezka Studio, Сербин",13.06 GB,59,63
16,Зарубежные фильмы,16 Май 23,"Air: Большой прыжок / Air (2023) WEB-DL 720p от селезень | P, A",3.49 GB,68,49
22,Зарубежные фильмы,16 Май 23,Подземелья и драконы: Честь среди воров / Dungeons & Dragons: Honor Among Thieves (2023) WEB-DLRip от MegaPeer | Jaskier,2.18 GB,261,51
23,Зарубежные фильмы,16 Май 23,Air: Большой прыжок / Air (2023) WEB-DLRip от MegaPeer | HDRezka Studio,1.46 GB,91,59
24,Зарубежные фильмы,16 Май 23,"Air: Большой прыжок / Air (2023) WEB-DL 1080p от New-Team | P, A",6.75 GB,363,217


In [ ]:
id = [244,245]
my_df.loc[id, 'href']

244    http://5.45.84.133/torrent/924752/star-control-origins-galactic-edition-v-1.62.366103-dlcs-2018-pc-repack-ot-fitgirl
245                   http://5.45.84.133/torrent/924389/resident-evil-4-deluxe-edition-build-11025382-dlcs-2023-pc-portable
Name: href, dtype: object